In [12]:
import os
import geopandas as gpd
import pandas as pd
from geopy.geocoders import Nominatim
import time
import folium
from folium.plugins import MarkerCluster
from shapely.wkt import loads  # To convert WKT strings to shapely geometry objects

In [13]:
# Create folder to save dataset
base_dir = '../../data/'
landing_dir = os.path.join(base_dir, 'landing')
raw_dir = os.path.join(base_dir, 'raw')

if not os.path.exists(raw_dir):
    os.makedirs(raw_dir)

subfolder = 'Foundation_Electricity_Infrastructure'

if not os.path.exists(os.path.join(raw_dir, subfolder)):
    os.makedirs(os.path.join(raw_dir, subfolder))

In [14]:
# read geojson file dataset
substations_path = f"{landing_dir}/{subfolder}/transmission_substations.geojson"
substations_gdf = gpd.read_file(substations_path)

# Observe dataset
print(substations_gdf.head(3))

# data structure
print(substations_gdf.info())

   objectid   uri feature_type          name operational_status  \
0         1  None   Substation  Homebush Bay        Operational   
1         2  None   Substation      Hillston        Operational   
2         3  None   Substation           Hay        Operational   

    feature_date      feature_source  attribute_date  \
0  1360195200000  Esri World Imagery   1360800000000   
1  1298419200000  Esri World Imagery   1360800000000   
2  1386979200000  Esri World Imagery   1360800000000   

                               attribute_source      custodian_agency  \
0  AEMO-Australian Energy Market Operators 2013  Geoscience Australia   
1  AEMO-Australian Energy Market Operators 2013  Geoscience Australia   
2  AEMO-Australian Energy Market Operators 2013  Geoscience Australia   

                                 custodian_licensing   loading_date  \
0  This material is released under the Creative C...  1602806400000   
1  This material is released under the Creative C...  1602806400000   


In [15]:
power_stations_path = f"{landing_dir}/{subfolder}/power_stations.geojson"
power_stations_gdf = gpd.read_file(power_stations_path)

print(power_stations_gdf.head(3))
print(power_stations_gdf.info())


   objectid   uri   feature_type                       name  \
0         1  None  Power Station                    Paloona   
1         2  None  Power Station  Bell Bay (Bell Bay Three)   
2         3  None  Power Station                  Trevallyn   

  operational_status  feature_date      feature_source  attribute_date  \
0        Operational  1.378080e+12  Esri World Imagery   1479168000000   
1        Operational  1.320365e+12  Esri World Imagery   1479168000000   
2        Operational  1.300320e+12  Esri World Imagery   1479168000000   

                               attribute_source      custodian_agency  ...  \
0                        Hydro Tasmania Website  Geoscience Australia  ...   
1  AEMO-Australian Energy Market Operators 2016  Geoscience Australia  ...   
2                        Hydro Tasmania Website  Geoscience Australia  ...   

  structuretype  operator                                  owner  \
0          None      None  Hydro-Electric Corporation (Tasmania)   
1

In [16]:
# Check if both GeoDataFrames have the same columns
columns_substations = set(substations_gdf.columns)
columns_power_stations = set(power_stations_gdf.columns)

if columns_substations == columns_power_stations:
    print("Both GeoDataFrames have the same features (columns).")
else:
    print("The GeoDataFrames have different features (columns).")
    # Print the differences
    print("Columns in gdf1 but not in gdf2:", columns_substations - columns_power_stations)
    print("Columns in gdf2 but not in gdf1:", columns_power_stations - columns_substations)

The GeoDataFrames have different features (columns).
Columns in gdf1 but not in gdf2: {'voltage'}
Columns in gdf2 but not in gdf1: {'embeddednetworkoperator', 'structuretype', 'generatornumber', 'primaryfueltype', 'generationmw', 'operator', 'owner', 'primarysubfueltype'}


In [17]:
# Perform union (concatenate rows), allowing for column mismatch
gdf_union = pd.concat([substations_gdf, power_stations_gdf], ignore_index=True)
gdf_union.head(3)

,objectid,uri,feature_type,name,operational_status,feature_date,feature_source,attribute_date,attribute_source,custodian_agency,...,address,geometry,structuretype,operator,owner,primaryfueltype,primarysubfueltype,generationmw,generatornumber,embeddednetworkoperator
0,1,None,Substation,Homebush Bay,Operational,1.360195e+12,Esri World Imagery,1360800000000,AEMO-Australian Energy Market Operators 2013,Geoscience Australia,...,Sydney Olympic Park New South Wales,POINT (151.07300 -33.83318),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,None,Substation,Hillston,Operational,1.298419e+12,Esri World Imagery,1360800000000,AEMO-Australian Energy Market Operators 2013,Geoscience Australia,...,Hillston New South Wales,POINT (145.52972 -33.51593),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,None,Substation,Hay,Operational,1.386979e+12,Esri World Imagery,1360800000000,AEMO-Australian Energy Market Operators 2013,Geoscience Australia,...,Hay New South Wales,POINT (144.87829 -34.48942),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
len(gdf_union)

1536

In [19]:
# Check for missing values in the dataset
print(substations_gdf.isnull().sum())


objectid                  0
uri                    1000
feature_type              0
name                      0
operational_status        0
feature_date              0
feature_source            0
attribute_date            0
attribute_source          0
custodian_agency          0
custodian_licensing       0
loading_date              0
class                     0
voltage                   0
address                  11
geometry                  0
dtype: int64


In [20]:
# Check for missing values in the dataset
print(power_stations_gdf.isnull().sum())

objectid                     0
uri                        536
feature_type                 0
name                         0
operational_status           0
feature_date                 1
feature_source               0
attribute_date               0
attribute_source             1
custodian_agency             0
custodian_licensing          0
loading_date                 0
class                        0
structuretype              528
operator                   310
owner                       18
primaryfueltype              6
primarysubfueltype         340
generationmw                 9
generatornumber            130
embeddednetworkoperator    302
address                     20
geometry                     0
dtype: int64


In [21]:
# Check for missing values in the dataset
print(gdf_union.isnull().sum())

objectid                      0
uri                        1536
feature_type                  0
name                          0
operational_status            0
feature_date                  1
feature_source                0
attribute_date                0
attribute_source              1
custodian_agency              0
custodian_licensing           0
loading_date                  0
class                         0
voltage                     536
address                      31
geometry                      0
structuretype              1528
operator                   1310
owner                      1018
primaryfueltype            1006
primarysubfueltype         1340
generationmw               1009
generatornumber            1130
embeddednetworkoperator    1302
dtype: int64


In [22]:
# Handling missing value

# Initialize geolocator with Nominatim
geolocator = Nominatim(user_agent="geojson_address_filler")

# Function to perform reverse geocoding
def reverse_geocode(point):
    try:
        # Extract latitude and longitude from geometry
        latitude = point.y
        longitude = point.x
        
        # Perform reverse geocoding to get the address
        location = geolocator.reverse((latitude, longitude), timeout=10)
        
        # Return the full address if available
        return location.address if location else 'Unknown'
    except Exception as e:
        return 'Error'

# Identify rows where 'address' is missing
missing_address_idx = gdf_union[gdf_union['address'].isnull()].index

# Fill missing 'address' using reverse geocoding on geometry
for idx in missing_address_idx:
    point = gdf_union.loc[idx, 'geometry']
    gdf_union.loc[idx, 'address'] = reverse_geocode(point)

    # Print the filled address for each index to check the result
    print(f"Filled address for index {idx}: {gdf_union.loc[idx, 'address']}")
    
    # Add a sleep to respect API rate limits
    time.sleep(1)



Filled address for index 491: Grand Central Shopping Centre, Margaret Street, Toowoomba City, Toowoomba, Toowoomba Regional, Queensland, 4350, Australia
Filled address for index 665: Dairy Road, Fyshwick, Canberra, District of Canberra Central, Australian Capital Territory, 2609, Australia
Filled address for index 771: McAskill Road, Willalo, The Regional Council of Goyder, South Australia, 5419, Australia
Filled address for index 772: Lincoln Gap, Pastoral Unincorporated Area, South Australia, 5715, Australia
Filled address for index 773: Murra Warra, Shire of Yarriambiack, Victoria, 3401, Australia
Filled address for index 774: Vances Crossing Road, Joel South, Shire of Northern Grampians, Victoria, Australia
Filled address for index 775: Spring Flat Road, Glenlofty, Shire of Pyrenees, Victoria, Australia
Filled address for index 776: Deer Park Terminal Station, 279-329, Christies Road, Ravenhall, Melbourne, City of Melton, Victoria, 3023, Australia
Filled address for index 778: Whit

In [23]:
# Filtering data, only need VIC data
gdf_filtered = gdf_union[gdf_union['address'].str.contains('Victoria', na=False)]
len(gdf_filtered)

161

In [24]:
unique_values = gdf_filtered['operational_status'].unique()
print(unique_values)

['Operational' 'Non-Operational']


In [25]:
# Filtering data to include only those with foundation electricity infrastructure that is currently operational
gdf_filtered = gdf_filtered[gdf_filtered['operational_status'] == 'Operational']
len(gdf_filtered)

160

In [26]:
gdf_filtered.head(3)

,objectid,uri,feature_type,name,operational_status,feature_date,feature_source,attribute_date,attribute_source,custodian_agency,...,address,geometry,structuretype,operator,owner,primaryfueltype,primarysubfueltype,generationmw,generatornumber,embeddednetworkoperator
33,34,None,Substation,Oaklands Hill Wind Farm,Operational,1.392077e+12,Esri World Imagery,1360800000000,AEMO-Australian Energy Market Operators 2013,Geoscience Australia,...,Glenthompson Victoria,POINT (142.55226 -37.68147),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,35,None,Substation,Mortons Lane Wind Farm,Operational,1.329955e+12,Esri World Imagery,1360800000000,AEMO-Australian Energy Market Operators 2013,Geoscience Australia,...,Woodhouse Victoria,POINT (142.46628 -37.83516),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,36,None,Substation,Ballarat North,Operational,1.320624e+12,Esri World Imagery,1360800000000,AEMO-Australian Energy Market Operators 2013,Geoscience Australia,...,Wendouree Victoria,POINT (143.84690 -37.53461),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
print(gdf_filtered.isnull().sum())

objectid                     0
uri                        160
feature_type                 0
name                         0
operational_status           0
feature_date                 0
feature_source               0
attribute_date               0
attribute_source             0
custodian_agency             0
custodian_licensing          0
loading_date                 0
class                        0
voltage                     86
address                      0
geometry                     0
structuretype              158
operator                   108
owner                       75
primaryfueltype             74
primarysubfueltype         134
generationmw                76
generatornumber             97
embeddednetworkoperator    126
dtype: int64


In [28]:
# delect some columns which contain large amount of missing values, and base on domain knowledge
columns_to_drop = ['objectid', 'uri', 'structuretype', 'operator', 'primarysubfueltype', 'embeddednetworkoperator', 'primaryfueltype', 'generationmw', 'generatornumber']
gdf_preprocessed = gdf_filtered.drop(columns=columns_to_drop)

In [29]:
print(gdf_preprocessed.shape)

(160, 15)


In [30]:
# save data
output_path = f"{raw_dir}/{subfolder}/foundation_electricity_infrastructure.csv"
gdf_preprocessed.to_csv(output_path, index=False)

In [32]:
# Visualize data
df = gdf_preprocessed  # Assuming gdf_preprocessed already has 'shapely' geometry points

# Convert the DataFrame to a GeoDataFrame (if needed, but skip this if gdf_preprocessed is already a GeoDataFrame)
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Create a folium map object, centered around a location in Australia
# Adjust the location and zoom_start based on your data's geographic range
m = folium.Map(location=[-37.68147, 142.55226], zoom_start=8)

# Create a MarkerCluster object for clustering the markers
marker_cluster = MarkerCluster().add_to(m)

# Add markers to the cluster for each substation in the GeoDataFrame
for _, row in gdf.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],  # Note: y is latitude, x is longitude
        popup=row['name'],  # Display the substation name in a popup
        tooltip=row['address']  # Show the address when hovering over the marker
    ).add_to(marker_cluster)

# Display the map in Jupyter Notebook or other environments
m

# Save the map to an HTML file for viewing in a web browser (uncomment to use)
# m.save("foundation_electricity_infrastructure_map.html")
